In [1]:
# 导包

In [2]:
import time
import numpy as np
import scipy.io as sio
from sklearn.datasets import load_svmlight_files
from sklearn.preprocessing import scale,LabelEncoder
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold, train_test_split
import numpy as np
import scipy.linalg as la
import scipy.io as sio
from sklearn.datasets import load_svmlight_files
from sklearn.preprocessing import scale,LabelEncoder
from sklearn.model_selection import train_test_split
from scipy.optimize import minimize

In [3]:
# 数据接口

In [4]:
def readPIE():
    data = sio.loadmat('D:\pytorch\统计学习方法实践\datasets\PIE.mat')
    X,y = data['fea'].astype(np.float64),data['gnd'].ravel()
    y = LabelEncoder().fit(y).transform(y).astype(int)
    #X = scale(X / X.sum(axis=1,keepdims=True))
    return X,y

def readBooks():
    file = 'D:\pytorch\统计学习方法实践\datasets\\books.svmlight'
    X, y = load_svmlight_files([file])
    X = X.toarray()
    y = LabelEncoder().fit(y).transform(y).astype(np.float64)
    #X = scale(X)
    return X,y

In [5]:
# knn函数

In [6]:
def KNN(test_data1,train_data_pca,train_label,k,p):
    sub_mat = train_data_pca - test_data1
    sub_mat = np.abs(sub_mat)
    distance = sub_mat ** p
    distance = np.sum(distance, axis=1)
    distance = distance ** (1.0 / p)
    distance_index = np.argsort(distance)
    class_count = [0 for i in range(68)]
    for i in range(k):
        label = train_label[distance_index[i]]
        class_count[label] += 1
    return np.argmax(class_count)

# 测试算法
def test(k, p):
    print("testing with K = %d and p = %d" % (k, p))
    m, n = np.shape(X_test_pca)
    correctCount = 0
    for i in range(m):
        test_data1 = X_test_pca[i, :]
        predict_label = KNN(test_data1, X_train_pca, y_train, k, p)
        true_label = y_test[i]
        if true_label == predict_label:
            correctCount += 1
    print("The accuracy is: %f" % (float(correctCount) / m))
    return float(correctCount) / m

In [7]:
# 算法

In [9]:
X,y = readPIE()
pred, preds = [], []
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

kf = KFold(n_splits=5, shuffle=True, random_state=2022)
start = time.perf_counter()

for trn_idx, tt_idx in kf.split(X):
    X_train, y_train, X_test, y_test = X[trn_idx], y[trn_idx], X[tt_idx], y[tt_idx]

    pca = PCA(n_components=68)
    pca.fit(X_train)
    X_train_pca = pca.transform(X_train)
    X_test_pca = pca.transform(X_test)

    # X_test_pca = X_test
    # X_train_pca = X_train

    # clf = KNeighborsClassifier(n_neighbors=5)
    # clf.fit(X_train_pca, y_train)
    #
    # #输出分类报告
    # print(classification_report(y_test, clf.predict(X_test_pca)))

    # for i in range(1, 10):
    #     for j in range(1, 5):
    #         pred.append(test(i, j))

    pred.append(test(1, 1))


end = time.perf_counter()

testing with K = 1 and p = 1
The accuracy is: 0.982009
testing with K = 1 and p = 1
The accuracy is: 0.973013
testing with K = 1 and p = 1
The accuracy is: 0.978979
testing with K = 1 and p = 1
The accuracy is: 0.983483
testing with K = 1 and p = 1
The accuracy is: 0.989489


In [10]:
print("每个分割的准确率为: {}".format(pred))
print("平均准确率为: {}".format(sum(pred) / len(pred)))
print("耗时为: {}".format(end - start))

每个分割的准确率为: [0.9820089955022488, 0.9730134932533733, 0.978978978978979, 0.9834834834834835, 0.9894894894894894]
平均准确率为: 0.9813948881415149
耗时为: 6.912548899999997


In [11]:
# 梯度下降和正规方程

In [12]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))


def regularized_cost(theta, X, y, lamda):
    thetaReg = theta[1:]
    first = (-y*np.log(sigmoid(X@theta))) + (y-1)*np.log(1-sigmoid(X@theta))
    reg = (thetaReg@thetaReg)*lamda / (2*len(X))
    return np.mean(first) + reg


def regularized_gradient(theta, X, y, l):
    thetaReg = theta[1:]
    first = (1 / len(X)) * X.T @ (sigmoid(X @ theta) - y)
    # 这里人为插入一维0，使得对theta_0不惩罚，方便计算
    reg = np.concatenate([np.array([0]), (l / len(X)) * thetaReg])
    return first + reg


def one_vs_all(X, y, l, K):
    all_theta = np.zeros((K, X.shape[1]))  # (10, 401)

    for i in range(1, K + 1):
        theta = np.zeros(X.shape[1])
        y_i = np.array([1 if label == i else 0 for label in y])
        print(y_i,theta)
        ret = minimize(fun=regularized_cost, x0=theta, args=(X, y_i, l), method='TNC',
                        jac=regularized_gradient, options={'disp': True})
        all_theta[i - 1, :] = ret.x

    return all_theta


def one_vs_all_normal(X, y, l, K):
    all_theta = np.zeros((K, X.shape[1]))  # (10, 401)

    for i in range(1, K + 1):
        theta = np.zeros(X.shape[1])
        y_i = np.array([1 if label == i else 0 for label in y])

        re = la.inv(np.dot(X.T,X) + 0.01 * np.eye(1025)).dot(X.T).dot(y_i)

        all_theta[i - 1, :] = re

    return all_theta


def predict_all(X, all_theta):
    h = sigmoid(X @ all_theta.T)  # 注意的这里的all_theta需要转置
    h_argmax = np.argmax(h, axis=1)
    h_argmax = h_argmax + 1

    return h_argmax

raw_X, raw_y = readPIE()
X = np.insert(raw_X, 0, 1, axis=1) # (5000, 401)
y = raw_y.flatten()

x_tr,x_te,y_tr,y_te = train_test_split(X,y,test_size=0.3,random_state=42)


all_theta = one_vs_all(x_tr, y_tr, 1, 68)
y_pred = predict_all(x_te, all_theta)
accuracy = np.mean(y_pred == y_te)
print ('accuracy = {0}%'.format(accuracy * 100))



all_theta_normal = one_vs_all_normal(x_tr, y_tr, 1, 68)
y_pred_normal = predict_all(x_te, all_theta_normal)
accuracy = np.mean(y_pred_normal == y_te)
print ('accuracy_normal = {0}%'.format(accuracy * 100))

[0 0 0 ... 0 0 0] [0. 0. 0. ... 0. 0. 0.]
[0 0 0 ... 0 0 0] [0. 0. 0. ... 0. 0. 0.]
[0 0 0 ... 0 0 0] [0. 0. 0. ... 0. 0. 0.]
[0 0 0 ... 0 0 0] [0. 0. 0. ... 0. 0. 0.]
[0 0 0 ... 0 0 0] [0. 0. 0. ... 0. 0. 0.]
[0 0 0 ... 0 0 0] [0. 0. 0. ... 0. 0. 0.]
[0 0 0 ... 0 0 0] [0. 0. 0. ... 0. 0. 0.]
[0 0 0 ... 0 0 0] [0. 0. 0. ... 0. 0. 0.]
[0 0 0 ... 0 0 0] [0. 0. 0. ... 0. 0. 0.]
[0 0 0 ... 0 0 0] [0. 0. 0. ... 0. 0. 0.]
[0 0 0 ... 0 0 0] [0. 0. 0. ... 0. 0. 0.]
[0 0 0 ... 0 0 0] [0. 0. 0. ... 0. 0. 0.]
[0 0 0 ... 0 0 0] [0. 0. 0. ... 0. 0. 0.]
[0 0 0 ... 0 0 0] [0. 0. 0. ... 0. 0. 0.]
[0 0 0 ... 0 0 0] [0. 0. 0. ... 0. 0. 0.]
[0 0 0 ... 0 0 0] [0. 0. 0. ... 0. 0. 0.]
[0 0 0 ... 0 1 0] [0. 0. 0. ... 0. 0. 0.]
[0 0 0 ... 0 0 0] [0. 0. 0. ... 0. 0. 0.]
[0 0 0 ... 0 0 0] [0. 0. 0. ... 0. 0. 0.]
[0 0 0 ... 0 0 0] [0. 0. 0. ... 0. 0. 0.]
[0 0 0 ... 0 0 0] [0. 0. 0. ... 0. 0. 0.]
[0 0 0 ... 0 0 0] [0. 0. 0. ... 0. 0. 0.]
[0 0 0 ... 0 0 0] [0. 0. 0. ... 0. 0. 0.]
[0 0 0 ... 0 0 0] [0. 0. 0. ... 0.